<div class="alert alert-block alert-info" style="background-color: #3933FF; border: 0px; -moz-border-radius: 10px; -webkit-border-radius: 10px;">
<h2 style="color: white">
Importing & Cleansing Data
</h2><br>
</div>

> ### The Excel file contains a journal entry and is generated by SAGA Accounting. 
> ### Perform some data cleansing to make it user friendly. 


---

In [ ]:
import pandas as pd

file_path = '/Users/andi/Desktop/CPD/REGISTRU JURNAL 29112023 1651.xls'
df = pd.read_excel(file_path)
df.head(20)


In [ ]:
df = pd.read_excel(file_path, header=8)

df.head(10)

In [ ]:
df.dropna(subset=['crt.'], inplace=True)
df.head(10)

In [ ]:
df.tail()


In [ ]:
filtered_df = df[df['Explicatie'].str.contains('google', case=False) & (df['debitor'] == 628)]
sum_debitor = filtered_df['debitor'].sum()
filtered_df, sum_debitor


In [ ]:
filtered_df = df[
    df['Explicatie'].str.contains('Google', case=False) 
    & (df['creditor'] != 512.1)
    ]
sum_debit = filtered_df['Debit'].sum()
sum_debit


----

<div class="alert alert-block alert-info" style="background-color: #3933FF; border: 0px; -moz-border-radius: 10px; -webkit-border-radius: 10px;">
<h2 style="color: white">
More Power
</h2><br>
</div>

> ### We connect extrenal libraries
> ### We bring the currency to the journal entry 

---

In [ ]:
from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

def get_historical_rates(currency_from, currency_to, start_year, end_year):
    c = CurrencyRates()
    
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)

    rates = {}
    current_date = start_date
    while current_date <= end_date:
        try:
            rate = c.get_rate(currency_from, currency_to, current_date)
            rates[current_date.strftime('%Y-%m-%d')] = rate
        except Exception as e:
            print(f"Could not fetch rate for date: {current_date.strftime('%Y-%m-%d')}. Error: {e}")
        current_date += timedelta(days=1)

    return rates

historical_rates = get_historical_rates('EUR', 'RON', 2023, 2023)
print(historical_rates)

In [ ]:

df['Exchange Rate'] = df['Data'].apply(lambda x: historical_rates.get(x.strftime('%Y-%m-%d'), None) if pd.notnull(x) else None)
df.head(10)

In [ ]:
filtered_df = df[
    df['Explicatie'].str.contains('Google', case=False) 
    & (df['creditor'] != 512.1)
    ]
filtered_df['Debit_with_exchange'] = filtered_df['Debit'] / filtered_df['Exchange Rate']
sum_debit = filtered_df['Debit_with_exchange'].sum()
sum_debit


In [ ]:
df['Debitor_First_3'] = df['debitor'].astype(str).str.slice(stop=3)
df['Creditor_First_3'] = df['creditor'].astype(str).str.slice(stop=3)
df.head(10)

In [ ]:
filtered_df = df[df['Debitor_First_3'] == '628']
filtered_df.head(10)

In [ ]:
unique_descriptions = filtered_df.groupby('Explicatie')['Debit'].sum().reset_index().rename(columns={'Debit': 'Total Debit Amount'})

unique_descriptions

---

<div class="alert alert-block alert-info" style="background-color: #3933FF; border: 0px; -moz-border-radius: 10px; -webkit-border-radius: 10px;">
<h2 style="color: white">
Plotting and Presentation
</h2><br>
</div>

> ### We can plot as in excel spreadsheets
> ### Here we are plotting the daily cash at bank

---

In [ ]:
import matplotlib.pyplot as plt

bank_df = df[df['debitor'].astype(str).str.startswith('512') | df['creditor'].astype(str).str.startswith('512')]

bank_df['Amount'] = bank_df.apply(lambda row: row['Debit'] if str(row['debitor']).startswith('512') else -row['Credit'], axis=1)

bank_df['Balance'] = bank_df['Amount'].cumsum()

plt.plot(bank_df['Balance'])
plt.xlabel('Transaction')
plt.ylabel('Balance')
plt.title('Bank Account Balance')
plt.show()

---

<div class="alert alert-block alert-info" style="background-color: #3933FF; border: 0px; -moz-border-radius: 10px; -webkit-border-radius: 10px;">
<h2 style="color: white">
Connecting a Generative AI Platform
</h2><br>
</div>

> #### We connect a Generative AI like OpenAI through LanghChain 
> #### We use a predefined Agent that execute python commands from natural language 
> #### This will allow us to explore data easily


---

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OpenAI API key not found. Please set it in your environment variables.")
openai.api_key = openai_api_key

In [ ]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0.5, model="gpt-4-1106-preview"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
agent.run("what is this dataframe and for what can be used?")

In [ ]:
agent.run("can you run a Benfords test?")

In [ ]:
df_journalEntry = df

<div class="alert alert-block alert-info">
    <b>💡 Example:</b> Now with <strong>more</strong> data sets, a journal entry and a trial balance.
</div>

In [ ]:
import pandas as pd

file_path = '/Users/andi/Desktop/CPD/Balanta de verificare 29112023 1653.xls'

df_trialBalance = pd.read_excel(file_path)

df_trialBalance.head(20)

In [ ]:
df_trialBalance = pd.read_excel(file_path, header=9)

df_trialBalance.head(10)

In [ ]:
df_trialBalance.drop(columns=['Unnamed: 2', 'Unnamed: 6'], inplace=True)
df_trialBalance.head(10)

In [ ]:
df_trialBalance.rename(columns=
                       {'Unnamed: 0': 'account', 
                        'Unnamed: 1': 'Description', 
                        'Debitoare': 'Opening Debit', 
                        'Creditoare': 'Opening Credit', 
                        'Debitoare.1': 'Transaction Debit', 
                        'Creditoare.1': 'Transaction Credit', 
                        'Debitoare.2': 'Total Transactions Debit', 
                        'Creditoare.2': 'Total Transactions Credit', 
                        'Debitoare.3': 'Closing Debit', 
                        'Creditoare.3': 'Closing Credit'
                        }, 
                       inplace=True)
df_trialBalance.head(10)

In [ ]:
df_trialBalance.dropna(subset=['account'], inplace=True)
df_trialBalance.head(10)

In [ ]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0.5, model="gpt-4-1106-preview"),
    [df_trialBalance, df_journalEntry],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
agent.run("what informations are in data frames? And how they ralate to eachother")

In [ ]:
agent.run("can you reconcile the information between the two data frames to see if are in agreement?")